In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

In [0]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
from numpy.random import seed

from inception_model_generator import MRNet_inc_model

import tensorflow as tf


#trainning on augmented balanced data with only the skewed class augmented
20 epochs


In [0]:
label_type = 'abnormal'
exam_view = 'sagittal'
model, cbs = MRNet_inc_model(1,0.00001,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos

class_weight = {0: 1, 1: neg/pos}
if neg < pos:
  class_weight = {0: pos/neg, 1: 1}

print(class_weight)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l ,{0:1,1:1}, batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l,{0:1,1:1} ,batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)


model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/"+label_type+"/"+exam_view+"/weights.74.hdf5")
model.fit(x=train_data_gen, validation_data = test_data_gen,epochs=88, initial_epoch = 74,verbose=True, class_weight=class_weight,callbacks=[cbs[0]],shuffle=False, use_multiprocessing=True, workers=6)

no dropout
no dropout
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 1)                 21804833  
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________
{0: 4.242268041237113, 1: 1}
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  train
Combination:  abnormal  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1017
input size:  (299, 299)
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  test
Combination:  abnormal  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  113
input size:  (299, 299)
Epoch 75/88
1017/1017 [==============================] - 553s 544ms/step - loss: 0.52

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 2 could not be retrieved. It could be because a worker has died.
  UserWarning)


1017/1017 [==============================] - 505s 497ms/step - loss: 0.4842 - tp: 7057.0718 - fp: 115.2665 - tn: 1716.5999 - fn: 660.0620 - accuracy: 0.9188 - precision: 0.9839 - recall: 0.9144 - auc: 0.9667 - val_loss: 0.3774 - val_tp: 7475.9292 - val_fp: 121.9823 - val_tn: 1820.1947 - val_fn: 695.8938 - val_accuracy: 0.9191 - val_precision: 0.9839 - val_recall: 0.9148 - val_auc: 0.9669

Epoch 00083: saving model to /content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/abnormal/sagittal/weights.83.hdf5
Epoch 84/88
1017/1017 [==============================] - 475s 467ms/step - loss: 0.4592 - tp: 7908.0107 - fp: 125.3422 - tn: 1925.7168 - fn: 719.9302 - accuracy: 0.9208 - precision: 0.9844 - recall: 0.9165 - auc: 0.9679 - val_loss: 1.3170 - val_tp: 8322.5137 - val_fp: 132.1416 - val_tn: 2027.2743 - val_fn: 762.0708 - val_accuracy: 0.9205 - val_precision: 0.9844 - val_recall: 0.9161 - val_auc: 0.9680

Epoch 00084: saving model to /content/gdrive/My Drive/Colab Notebooks/MRNet

# **testing on validation dataset**

In [0]:
label_type = 'abnormal'
exam_view = 'sagittal'
model, cbs = MRNet_inc_model(1,0,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/"+label_type+"/"+exam_view+"/weights.88.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])

neg = total - pos
class_count_valid = {0: neg, 1: pos}

batch_size = 1
print(model.metrics_names)
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , batch_size=1,model="inception",data_type='valid', shuffle=False,label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
d = model.evaluate_generator(valid_data_gen, total, verbose=1)
print(d)
model.predict_generator(valid_data_gen, 5, verbose=1)

no dropout
no dropout
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_3 (MR (None, 1)                 21804833  
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________
['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']
Initializing Data Generator:
factor :  3.8
model:  inception
data type:  valid
Combination:  abnormal  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (299, 299)
120/120 [==============================] - 72s 601ms/step
[0.20557881891727448, 44.44166564941406, 4.858333110809326, 7.741666793823242, 3.4583332538604736, 0.8511635065078735, 0.874819815158844, 0.9327707886695862, 0.819103479385376]
5/5 [==============================] - 3s 626ms/ste

array([[0.99288404],
       [0.57753485],
       [0.9446083 ],
       [0.28364035],
       [0.9976029 ]], dtype=float32)